# Problema
Um painel solar térmico (superfície negra ideal) possui 3 vidros completamente transparentes para a radiação solar, e com absorvidade $a$ a para a radiação infravermelha. A Figura 1 mostra o diagrama de fluxos radiatvos desse painel.

<img src="imgs/painel.png" width="400px" style="display: block; margin:auto" />

Admitindo que o sistema se encontra em equilíbrio radiativo (não existindo outros fluxos de energia para o exterior), note que os fluxos de radiação infravermelha emitida $(E0, E1, E2, E3)$ satisfazem o sistema de 4 equações:

$$
\begin{align*}
\begin{cases}
-E_0 + E_1 + (1-a)E_2 + (1-a)^2E_3+E_S &= 0 \\
aE_0 - 2E_1 + aE_2 + a(1-a)E_3 &= 0 \\
a(1-a)E_0 + aE_1 - 2E_2 + aE_3 &= 0 \\
a(1-a)^2E_0 + a(1-a)E_1 + aE_2 - 2E_3 &= 0
\end{cases}
\end{align*}$$

Calcule os fluxos emitidos pelas 4 superfícies, resolvendo o sistema de equações para o caso $a=0.8$, $E_S=500Wm^-2$. 

O sistema de equações lineares deve ser resolvido utilizando o método de Eliminação Gaussiana, sem e com pivotamento. 

# Solução

### Importação das bibliotecas

Para resolver o problema proposto, é útil que importemos funcionalidades de bibliotecas do Python que facilitam o processo de solução.
O [NumPy](https://numpy.org/) é uma biblioteca de código aberto usada praticamente para todo problema matemático, pois providencia uma gama de funções e estruturas diversas.

In [88]:
import numpy as np # Importa o módulo numpy e atribui o apelido de np para fácil referência posterio

from IPython.display import display, Latex, Markdown # Funções de utilidade do IPython para desenhar cells do Jupyter Notebook.

from furg_imef_verificador_respostas import Verificador

def verificar_resposta(r):
    verificador = Verificador()
    verificador.verificar_resposta(r)


## Definição das informações do problema

Substituindo no sistema de equações os valores para $a = 0.8$ e $E_S = 500$, obtemos o sistema de equações

\begin{align*}
\begin{cases}
-E_0 + E_1 + 0.2E_2 + 0.04E_3    &= -500 \\
0.8E_0 - 2E_1 + 0.8E_2 + 0.16E_3 &= 0 \\
0.16E_0 + 0.8E_1 - 2E_2 + 0.8E_3 &= 0 \\
0.032E_0 + 0.16E_1 + 0.8E_2 - 2E_3 &= 0
\end{cases}
\end{align*}

De onde pode-se extrair a matriz de coeficientes e vetor de variáveis dependentes,

\begin{align*}
A &=
\begin{bmatrix}
-1 & 1 & 0.2 & 0.04 \\
0.8 & -2 & 0.8 & 0.16 \\
0.16 & 0.8 & -2 & 0.8 \\
0.032 & 0.16 & 0.8 & -2
\end{bmatrix}
\end{align*}

\begin{align*}
b &=
\begin{bmatrix}
-500 \\ 0 \\ 0\\ 0
\end{bmatrix}
\end{align*}

Em Python:

In [89]:
a  = 0.8
Es = 500
A = np.array([
    [-1, 1, (1-a), (1-a)**2],
    [a, -2, a, a*(1-a)],
    [a*(1-a), a, -2, a],
    [a*(1-a)**2, a*(1-a), a, -2]
])
b = [-Es, 0, 0, 0]

display(A, b)

array([[-1.   ,  1.   ,  0.2  ,  0.04 ],
       [ 0.8  , -2.   ,  0.8  ,  0.16 ],
       [ 0.16 ,  0.8  , -2.   ,  0.8  ],
       [ 0.032,  0.16 ,  0.8  , -2.   ]])

[-500, 0, 0, 0]

## Eliminação Gaussiana **sem** pivotamento

### 1) Matriz triangular superior

O primeiro passo da eliminação gaussiana sem pivotamento compreende em calcular a _[matriz triangular superior](https://pt.wikipedia.org/wiki/Matriz_triangular)_ da matriz. Um algoritmo simples para esse passo envolve somar toda linha $a_i$ abaixo da linha $a_j$ por um múltiplo $\gamma$ de forma que:

$$
    a_{i,j} + \gamma a_{j,j} = 0 \Rightarrow
    a_{i} := a_{i} + \gamma a_{j} \hspace{0.25cm} \forall i > j
$$

Onde $j$ varia de $1$ a $m$ numa matriz $A_{m \times n}$. 


In [90]:

def matriz_triangular_superior(M, ignorar_ultima_coluna=True):
    """
        Calcula a matriz triangular superior da matriz M utilizando
        um algoritmo simples da eliminação de Gauss

        Parâmetros
        ----------
        M : np.array
            Matriz inicial.
        ignorar_ultima_coluna : bool
            Determina se devemos ignorar a última coluna da matriz.

        Retorno 
        ----------
        np.array
            Matriz na forma triangular superior.
    """
    
    # Criar cópia da matriz expandida
    M = M.copy()
    m = M.shape[0]
    n = M.shape[1] # - (1 if ignorar_ultima_coluna else 0)
    n = min(n, m)
    
    # Para cada coluna na matriz
    for j in range(n):
        # Selecionamos o pivô, como não há pivotamento, selecionamos os valores da diagonal
        ajj = M[j, j]
        
        # Para cada linha i abaixo de j
        for i in range(j+1, m):
            # Selecionamos o valor da linha i na coluna j (abaixo de ajj)
            aij = M[i, j]
            
            # Calculamos o múltiplo necessário do pivô ajj para que
            # x * ajj + aij = 0
            coeficiente = abs(aij) / ajj * -np.sign(aij)
            
            # Multiplicamos a linha inteira pelo coeficiente calculado
            A_i = M[j,:]*coeficiente
            
            # Atualizamos o valor da linha para o novo valor, que 
            # deve ser 0 na posição aij
            M[i, :] = M[i,:]+A_i
            
    return M

#### Testando a função

In [91]:
# Você não precisa executar essa célula.
M1 = np.array([[1, 2, 3], [3, 2, 1], [2, 1, 3]])
M2 = matriz_triangular_superior(M1, True)


display(M1, M2)

array([[1, 2, 3],
       [3, 2, 1],
       [2, 1, 3]])

array([[ 1,  2,  3],
       [ 0, -4, -8],
       [ 0,  0,  3]])

Tendo uma matriz nesse formato, já podemos resolver o problema. Tudo que precisamos é definir uma função que resolve matrizes de coeficientes expandidas com $b$ na última coluna. Para esse passo, no entanto, é preciso reduzir à matriz triangular a matriz expandida $[A|b]$.

In [92]:
def resolver_U(U, b):
    """
    Resolve um sistema de equações cuja matriz de coeficientes é triangular superiora.
    """
    x = np.zeros(U.shape[1])
    for i in reversed(range(U.shape[0])):
        x[i]= ( b[i] - np.sum([U[i,j]*x[j] for j in range(i+1, U.shape[0])]) ) / U[i,i]
    return x

In [99]:
# Expandir matriz de coeficientes com b.
Ae = np.insert(A, A.shape[1], b, axis=1)
Au = matriz_triangular_superior(Ae, True)

display(np.round(Au, 2))
x = resolver_U(Au[:, :-1], Au[:, -1])

# Verificamos se nosso vetor de variáveis independentes
# resultam em b
display('Ax = ')
display((A@x).round())

# Ou podemos utilizar o verificador de respostas
verificar_resposta(x)

array([[-1.0e+00,  1.0e+00,  2.0e-01,  4.0e-02, -5.0e+02],
       [ 0.0e+00, -1.2e+00,  9.6e-01,  1.9e-01, -4.0e+02],
       [ 0.0e+00,  0.0e+00, -1.2e+00,  9.6e-01, -4.0e+02],
       [ 0.0e+00, -0.0e+00,  0.0e+00, -1.2e+00, -4.0e+02]])

'Ax = '

array([-500.,   -0.,    0.,   -0.])

Resposta correta!


> **NOTA** À primeira vista o vetor resultante de $Ax$ pode parecer não resultar em $b$, mas é necessário ressaltar que metodos computacionais para a resolução de sistemas em $R$ possuem graus de [imprecisão numérica](https://pt.wikipedia.org/wiki/V%C3%ADrgula_flutuante). Apesar de irrelevante com operações únicas, algoritmos com muitas operações acabam criando um efeito composto de imprecisões e o resultado pode cair consideravelmente longe do esperado. No caso desse resultado específico, pode-se observar que a imprecisão é muito pequena, mas ainda se faz presente. Onde intuitivamente esperaríamos o valor $0$, obtemos valores como $-2.27373675 \times 10^{-13}$, $-1.13686838 \times 10^{-13}$, etc. Por isso, geralmente arredondamos nossas respostas.

In [69]:
# Arredondando o resultado, podemos verificar que os dois valores
# são praticamente idênticos.
display(np.round(A@x,2))
display(b)

array([-500.,   -0.,    0.,   -0.])

[-500, 0, 0, 0]

### 2) Redução à forma de Gauss-Jordan
É possível ir além com o algoritmo e obter uma matriz diagonal da nossa matriz inicial. Um algoritmo para reduzir a matriz triangular a uma matriz diagonal é simples e consiste na aplicação do algoritmo anterior em ordem inversa. Isto é, ao invés descermos a matriz diagonalmente e reduzirmos os valores abaixo de $a_{j,j}$ à zero, subimos no sentido contrário e reduzimos os valores acima de $a_{j,j}$ à zero. 

$$
    a_{i,j} + \gamma a_{j,j} = 0 \Rightarrow
    a_{i} := a_{i} + \gamma a_{j} \hspace{0.25cm} \forall i < j
$$

Onde agora $j$ varia de $m$ a $1$.

Se mantermos a matriz de variáveis dependentes na última coluna, podemos usá-la como $x$ na equação e obter a resposta.

In [70]:
def matriz_triangular_para_diagonal(M):
    # Criar cópia da matriz original
    M = M.copy()
    
    # Dimensões da matriz
    m = M.shape[0]
    n = M.shape[1]
    
    # Iterar sobre toda coluna j da matriz, começando
    # pelo valor mais baixo da sua diagonal.
    # Aqui, a função reversed inverte o intervalo retornado
    # por range.
    for j in reversed(range(m)):
        
        # Selecionar o valor na matriz na posição (j, j),
        # ou seja, na sua diagonal
        ajj = M[j, j]
        
        # Multiplicar a linha por 1/ajj, para que
        # o valor em (j,j) = 1
        M[j] = M[j]*(1/ajj)
        
        # Atualizar ajj para o valor normalizado
        ajj = M[j,j]
        
        # Observe que o intervalo retornado pela função range é 
        # aberto em relação a seu limite superior j, assim,
        # o retorno da função reversed resulta em (j-0]
        # logo, i varia de 0 a j-1 nesse loop.
        for i in reversed(range(j)):  
        # Para cada valor no intervalo (j-0]
        
            # Selecionamos o valor aij
            aij = M[i, j]
            
            # Calculamos gamma, ou o coeficiente de ajj para que
            # aij + ajj*gamma = 0
            coeficiente = abs(aij) / ajj * -np.sign(aij)
            
            # aj * gamma
            A_i = M[j]*coeficiente
            
            # Atualizar linha i com o novo valor
            # ai + aj*gamma = 0
            M[i] = M[i]+A_i
    
    return M

In [71]:
Ad = matriz_triangular_para_diagonal(Au)
x = Ad[:, -1]

In [72]:
display('Aplicando a função a Au obtemos:')
display(Ad.round())
display('x = ')
display(x)

'Aplicando a função a Au obtemos:'

array([[ 1.00e+00, -0.00e+00, -0.00e+00, -0.00e+00,  1.50e+03],
       [-0.00e+00,  1.00e+00, -0.00e+00, -0.00e+00,  8.67e+02],
       [-0.00e+00,  0.00e+00,  1.00e+00, -0.00e+00,  6.00e+02],
       [-0.00e+00,  0.00e+00, -0.00e+00,  1.00e+00,  3.33e+02]])

'x = '

array([1500.        ,  866.66666667,  600.        ,  333.33333333])

In [73]:
verificar_resposta(x)

Resposta correta!


## Pivotamento parcial e completo

O algoritmo de eliminação Gaussiana sem pivotamento pode resultar em problemas de instabilidade numérica em função do cálculo do coeficiente que definimos como $\gamma$. Como buscamos $$ a_{i,j} + \gamma a_{j,j} = 0 $$ então $$ \gamma = -\frac{a_{i,j}}{a_{j,j}} $$

Se $a_{j,j}$ é um valor muito pequeno, ou $0$, a divisão pode amplificar a imprecisão numérica, ou resultar numa indefinição. Para esses casos, o pivotamento ajuda a mitigar o problema reordenando a matriz em cada passo da eliminação Gassiana, para que $a_{j,j}$ seja o maior valor da matriz $a_{k, l}$ onde $k > j$, $l > j$.

Para facilitar o processo primeiro implementamos funções para trocar duas linhas e duas colunas de uma matriz.

In [74]:
def trocar_linha(A, i, j):
    """
    Troca a linha i pela linha j de uma matriz A
    """
    # É importante copiar a linha, pois caso contrário
    # linha_i é uma referência à matriz original,
    # e alterar A[i] alteraria a linha_i também.
    linha_i = A[i,:].copy()
    A[i,:] = A[j,:]
    A[j,:] = linha_i
    return A

def trocar_coluna(A, i, j):
    """
    Troca a coluna i pela coluna j de uma matriz A
    """
    linha_i = A[:, i].copy()
    A[:, i] = A[:, j]
    A[:, j] = linha_i
    return A

In [75]:
m = np.array([[1, 2, 3], [2, 3, 1], [3, 1, 2]])
display(m)
display('Trocando coluna 1 pela coluna 2:')
display(trocar_coluna(m, 0, 1))
display('Trocando linha 2 pela linha 3:')
display(trocar_linha(m,1,2))

array([[1, 2, 3],
       [2, 3, 1],
       [3, 1, 2]])

'Trocando coluna 1 pela coluna 2:'

array([[2, 1, 3],
       [3, 2, 1],
       [1, 3, 2]])

'Trocando linha 2 pela linha 3:'

array([[2, 1, 3],
       [1, 3, 2],
       [3, 2, 1]])

### Pivotamento Parcial

O pivotamento parcial é simples pois não envolve a reordenação das variáveis independentes dentro da matriz de coeficientes, e serve para a maioria dos casos. Para cada coeficiente na diagonal principal da matriz, escolhemos o maior valor na sua coluna entre ele e os valores abaixo.

In [76]:
def pivotamento_parcial(M, i):
    valores_abaixo_do_pivo = M[i:, i]
    indice_novo_pivo = np.argmax(np.abs(valores_abaixo_do_pivo))
    linha = i+indice_novo_pivo
    trocar_linha(M, i, linha)
    return M

In [77]:
def matriz_triangular_superior(M, pivotamento=False, log=False):
    """
        Calcula a matriz triangular superior da matriz M utilizando
        um algoritmo simples da eliminação de Gauss
    """
    # Criar cópia da matriz expandida
    A_new = M.copy()
    m = A_new.shape[0]
    n = A_new.shape[1]
    
    # Para cada coluna - 1 na matriz (ignoramos a coluna das variáveis dependentes)
    for j in range(n-1):
        
        if log:
            display('Estado atual da matriz:')
            display(A_new.round(1))
        
        # Maximizar pivô
        if pivotamento:
            pivotamento_parcial(A_new, j)
            
        if log:
            display('Resultado pivotamento:')
            display(A_new.round(1))
        
        ajj = A_new[j, j]
        
        # Para cada linha i abaixo de j
        for i in range(j+1, m):
            # Selecionamos o valor da linha i na coluna j (abaixo de ajj)
            aij = A_new[i, j]
            
            # Calculamos o múltiplo necessário do pivô ajj para que
            # x * ajj + aij = 0
            coeficiente = abs(aij) / ajj * -np.sign(aij)
            
            # Multiplicamos a linha inteira pelo coeficiente calculado
            A_i = A_new[j,:]*coeficiente
            
            # Atualizamos o valor da linha para o novo valor, que 
            # deve ser 0 na posição aij
            A_new[i] = A_new[i,:]+A_i
            
        if log:
            display('Resultado:')
            display(A_new.round(1))

    return A_new


In [78]:
Ats = matriz_triangular_superior(Ae, True)

# Podemos resolver como sistema de equações
x = resolver_U(Ats[:, :-1], Ats[:, -1])

# Ou reduzir a matriz escalonada para a forma de Gauss-Jordan.
x = matriz_triangular_para_diagonal(Ats)[:, -1]

Utilizando a função acima obtemos a matriz:

In [79]:
display(Ats.round(2))

array([[-1.0e+00,  1.0e+00,  2.0e-01,  4.0e-02, -5.0e+02],
       [ 0.0e+00, -1.2e+00,  9.6e-01,  1.9e-01, -4.0e+02],
       [ 0.0e+00,  0.0e+00, -1.2e+00,  9.6e-01, -4.0e+02],
       [ 0.0e+00, -0.0e+00,  0.0e+00, -1.2e+00, -4.0e+02]])

Podemos então resolver o sistema de equações e obter o vetor

In [80]:
display(x.reshape(x.shape[0], 1).round(2))

array([[1500.  ],
       [ 866.67],
       [ 600.  ],
       [ 333.33]])

In [81]:
verificar_resposta(x)

Resposta correta!


### Pivotamento Total

O pivotamento total além de procurar abaixo do pivô atual um valor maior para ser novo pivô, procura também em outras colunas. No entanto, esse processo aumenta a complexidade do algoritmo por criar a necessidade de mantermos uma matriz de permutação que registra as trocas de colunas que fizemos.

O motivo dessa complexidade é que, se descrevermos o sistema a ser solvido como

$$
\begin{cases}
a_{1,1}x_1 + a_{1,2}x_2 + \dots + a_{1,m} = b_1 \\
a_{2,1}x_1 + a_{2,2}x_2 + \dots + a_{2,m} = b_2 \\
\dots \\
a_{n,1}x_1 + a_{n,2}x_2 + \dots + a_{n,m} = b_n
\end{cases}
$$

A troca de duas linhas não altera a ordem das variáveis independentes no nosso vetor $x$

$$
\begin{cases}
a_{2,1}x_1 + a_{2,2}x_2 + \dots + a_{2,m} = b_2 \\
a_{1,1}x_1 + a_{1,2}x_2 + \dots + a_{1,m} = b_1 \\
\dots \\
a_{n,1}x_1 + a_{n,2}x_2 + \dots + a_{n,m} = b_n
\end{cases}
$$

Enquanto a troca de duas colunas resulta em

$$
\begin{cases}
a_{1,2}x_2 + a_{1,1}x_1 +  \dots + a_{1,m} = b_1 \\
a_{2,2}x_2 + a_{2,1}x_1 +  \dots + a_{2,m} = b_2 \\
\dots \\
a_{n,2}x_2 + a_{n,1}x_1 +  \dots + a_{n,m} = b_n
\end{cases}
$$

Por fim, temos que nosso vetor de variáveis independentes pode ser expressado como

$$
x = Q_1Q_2 \dots Q_n y  
$$

In [82]:
# Um cuidado necessário nessa implementação deve ser o de não considerar
# a última coluna na busca pelo maior valor da matriz, pois esta contém
# os valores de variáveis dependentes.
def pivotamento_total(M, indice):
    M_ = M[indice:, indice:-1]
    Q  = np.eye(M.shape[0])

    # Obtemos o índice do maior valor da matriz, dentro dos valores
    # disponíveis para escolha, e calculamos as coordenadas
    # do valor no formato da matriz.
    v = np.argmax(np.abs(M_)) # Argmax retorna o índice de um array de dimensões 1 x m*n
    i = int(np.floor(v/M_.shape[1]))
    j = int(v - i*M_.shape[1])

    # Movemos o pivô para a posição correta na matriz.
    trocar_linha(M, indice, indice+i)
    trocar_coluna(M, indice, indice+j)
    
    # Alteramos a matriz de permutação da mesma forma.
    trocar_coluna(Q, indice, indice+j)

    return Q


In [83]:

def matriz_triangular_superior(M, pivotamento=False, log=False):
    # Criar cópia da matriz expandida
    A_new = M.copy()
    m = A_new.shape[0]
    n = A_new.shape[1]

    Q = np.eye(m)
    
    # Para cada coluna - 1 na matriz (ignoramos a coluna das variáveis dependentes)
    for j in range(n-1):
        
        if log:
            display('Estado atual da matriz:')
            display(A_new.round(1))
        
        # Maximizar pivô
        if pivotamento:
            Qi = pivotamento_total(A_new, j)
            Q = Q @ Qi
            
        if log:
            display('Resultado pivotamento:')
            display(A_new.round(1))
        
        ajj = A_new[j, j]
        
        # Para cada linha i abaixo de j
        for i in range(j+1, m):
            # Selecionamos o valor da linha i na coluna j (abaixo de ajj)
            aij = A_new[i, j]
            
            # Calculamos o múltiplo necessário do pivô ajj para que
            # x * ajj + aij = 0
            coeficiente = abs(aij) / ajj * -np.sign(aij)
            
            # Multiplicamos a linha inteira pelo coeficiente calculado
            A_i = A_new[j,:]*coeficiente
            
            # Atualizamos o valor da linha para o novo valor, que 
            # deve ser 0 na posição aij
            A_new[i] = A_new[i,:]+A_i
            
        if log:
            display('Resultado:')
            display(A_new.round(1))

    return A_new, Q


In [84]:
Au, Q = matriz_triangular_superior(Ae, True)

In [85]:
display('Au = ')
display(Au.round(0))
display('Q = ')
display(Q)

'Au = '

array([[  -2.,    0.,    1.,    1.,    0.],
       [   0.,   -2.,    1.,    0.,    0.],
       [   0.,    0.,   -1.,    1.,    0.],
       [   0.,    0.,    0.,   -0., -500.]])

'Q = '

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

Obtemos a resposta resolvendo o sistema de equações como antes, mas agora multiplicamos o vetor $x$ resultante pela matriz de permutação calculada.

In [86]:
# Podemos resolver por substituição
x = resolver_U(Au[:, :-1], Au[:, -1])

# Ou reduzir a matriz.
x = matriz_triangular_para_diagonal(Au)[:, -1]

# E então reordenar as variáveis independentes.
x = Q@x

verificar_resposta(x)

Resposta correta!
